# Measure dv/v at single stations using trace-streching method
Notebook written by Cody A. Kupres at Purdue University

Reference: Kupres, C. A., Yang, X, Haney M., Roman, D. C. (in review) Sustained co-eruptive increase in seismic velocity below Great Sitkin Volcano due to magma extrusion. Geophysical Research Letters


In [ ]:
#IMPORT SECTION
import sys,os,time,glob,scipy
import obspy
import pyasdf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from seisgo import noise, monitoring, utils
import seisgo.plotting as sp
from seisgo.stacking import seisstack
import matplotlib.dates as mdates
import seisgo.downloaders as dld
import pickle

In [ ]:
# GREAT SITKIN ISLAND
#####FILE LOCATIONS#####
rootpath = "/volcanodvv" # roothpath for the project
maindirec  = '/volcanodvv/data_greatsitkin' #main project folder
direc  = os.path.join(rootpath,'data_greatsitkin/MERGED_PAIRS')
figdir = os.path.join('Figures')
dvvdir  = os.path.join(maindirec,'dvv')
if not os.path.isdir(dvvdir):os.makedirs(dvvdir)

######SET PARAMTERS######
save = True                                 #Save?
format = 'png'                              #save format
allfreq = True
onelag = True                               #Make Measurement one lag or two
norm_flag = True                            #Whether to normalize the cross-correlation waveforms
vmin = 1                                    #minimum velocity of the direct waves ->  #start of the coda waveforms
nproc=20                                      #number of processors to use.
stack_method = 'robust'                     #which stacked data to measure dispersion info
Method = 'ts'
freqlst = [[0.1,0.4],[0.4,1],[1,2],[2,3]]                  #frequency

#####STATION#####
sta_list = ['AV.GSSP','AV.GSCK','AV.GSMY','AV.GSTD','AV.GSTR']
cc_complist = ['EE','NN','ZZ','EZ','EN','NZ']             #Which components to use (EZ, EN, NZ)

# Starting and end dates for study period
sdate="2019_07_01"
edate="2023_07_31"

#Coda window length (1-13s)
offset = 1
coda_wlen = 12

In [ ]:
print('Timer on!')
print()
t0=time.time()

for sta in sta_list:
    for comp in cc_complist:
        for freq in freqlst:
            offset = 1                            #offset
            coda_wlen = varpar                    #window length in seconds for the coda waves 

            freq = freq
            source = sta
            receiver = source
            cc_comp = comp

            #file handling#
            key=source+'_'+receiver
            ccfile=sorted(glob.glob(os.path.join(direc,source,'*'+receiver+'*.h5')))[0]
            corrdata0=noise.extract_corrdata(ccfile,pair=source+'_'+receiver,comp=cc_comp)
#                 lat = corrdata0.lat
#                 lon = corrdata0.lon

            #print(corrdata0['AV.GSSP_AV.GSSP']['EZ'].data[0])

            ##########NaN Slice Removal#####################
            newcorrdata = corrdata0.copy()

            ncorrdata = newcorrdata[key][cc_comp]

            newtime = []
            newdata= [] 

            for i,d in enumerate(ncorrdata.data): 
                if not np.isnan(d).any(): 
                    newtime.append(ncorrdata.time[i])
                    newdata.append(d)

            ncorrdata.time = np.array(newtime)
            ncorrdata.data = np.array(newdata)
            ############################################


            ##REFERENCE TRACE DATA
            #cdataref=corrdata[key][cc_comp]
            cdataref=ncorrdata.copy()
            sdateref="2019_07_05"
            edateref="2020_02_01"
            cdataref.subset(starttime=sdateref,endtime=edateref, overwrite = True)
            ref=seisstack(cdataref.data,method=stack_method)


            ##FULL DATA SET DURATION##
            #cdata=corrdata[key][cc_comp]
            cdata=ncorrdata
            cdata.subset(starttime=sdate,endtime=edate, overwrite = True)

            #HIGH FREQUENCY (FULL DURATION)
            #freq = freqrange                             #targeted frequency band for monitoring
            win_len0 = 3600*12                           #Window length i.e. Resolution
            dv_max = 8/100                             #limit for the dv/v
            cdata.plot(freqmin=freq[0],freqmax=freq[1], lag = coda_wlen+5, stack_method = stack_method)
            t1=time.time()
            filename = 'dvv.'+source+'.'+receiver+'.'+cc_comp+'.'+str(freq[0])+'-'+str(freq[1])+'Hz'+'.'+Method+'.pk'
            print('saving as: '+filename)
            dvvdata=monitoring.get_dvv(cdata,freq,coda_wlen,resolution=win_len0,vmin=vmin,ref = ref, method = Method, \
                                                whiten ='no',whiten_pad= 100,whiten_smooth=20,\
                                                stack_method=stack_method,plot=True,nproc=nproc, save=True, format='pickle',\
                                                outdir=dvvdir,outfile = filename, dvmax=dv_max, offset=offset)
            print("with %d processors, took %f seconds"%(nproc,time.time()-t1))

print('=============================')
print('Time! Total Time: '+str(time.time()-t0))